In [24]:
# Installing offline dependencies
!pip install -U --no-deps /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install -U --no-deps /kaggle/input/datasets-214/datasets-2.14.5-py3-none-any.whl

Processing /kaggle/input/faiss-gpu-173-python310/faiss_gpu-1.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/datasets-214/datasets-2.14.5-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4


In [6]:
!cp /kaggle/input/86-2-with-only-270k-articles/datasets-2.14.4-py3-none-any.whl /kaggle/working
!pip install  /kaggle/working/datasets-2.14.4-py3-none-any.whl
!cp /kaggle/input/backup-806/util_openbook.py .

Processing ./datasets-2.14.4-py3-none-any.whl
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [ ]:
!pip install -U sentence_transformers

### sentence transformer 저장하기

In [11]:
from transformers import AutoTokenizer, AutoModel
from sentence_transformers import SentenceTransformer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [12]:
## sentence-transformers/all-MiniLM-L6-v2 모델 불러오기 

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

In [13]:
## sentence-transformers/all-MiniLM-L6-v2 모델 저장하기

tokenizer.save_pretrained("/kaggle/working/all-MiniLM-L6-v2")
model.save_pretrained("/kaggle/working/all-MiniLM-L6-v2")

### wiki dataset 불러오기

In [16]:
!cp -r /kaggle/input/all-paraphs-parsed-expanded /kaggle/working/

In [17]:
## wiki dataset 불러오기 

from datasets import load_dataset, load_from_disk

paraphs_parsed_dataset = load_from_disk("/kaggle/working/all-paraphs-parsed-expanded")
modified_texts = paraphs_parsed_dataset.map(lambda example:
                                             {'temp_text':
                                              f"{example['title']} {example['section']} {example['text']}".replace('\n'," ").replace("'","")},
                                             num_proc=2)["temp_text"]

#0:   0%|          | 0/1050640 [00:00<?, ?ex/s]

#1:   0%|          | 0/1050639 [00:00<?, ?ex/s]

In [19]:
len(modified_texts)

2101279

In [21]:
modified_texts[444]

'Cetrimonium bromide Nanoparticle synthesis Surfactants play a key role in nanoparticle synthesis by adsorbing to the surface of the forming nanoparticle and lowering its surface energy. Surfactants also help to prevent aggregation (e.g. via DLVO mechanisms).'

### Faiss index 만들기

In [25]:
import faiss
from faiss import write_index, read_index

In [47]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
vectors = encoder.encode(modified_texts)

Batches:   0%|          | 0/65665 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [37]:
vector_dimension = vectors.shape[1]
index = faiss.IndexFlatL2(vector_dimension)
faiss.normalize_L2(vectors)
index.add(vectors)

In [ ]:
write_index(index, "/kaggle/working/all-MiniLM-L6-v2/faiss.index")

### index search 해보기

In [38]:
import numpy as np

search_text = 'natural language processing'
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [43]:
distances, idx = index.search(_vector, k=3)

In [44]:
idx

array([[92, 89,  3]])